# Experiments on the ACAS Xu benchmarks
This is an extended version of the ACAS Xu notebook provided in https://github.com/sputot/DSZAnalysis/. Our contribution is the empirical propagation approach for property satisfaction. 

The networks (in .nnet format) and properties are taken from https://github.com/guykatzz/ReluplexCav2017/tree/master/nnet 
(also available at onnx format with properties at https://github.com/stanleybak/vnncomp2021/tree/main/benchmarks/acasxu) 

Convert in Julia file by jupyter nbconvert --to script ACASXu.ipynb

In [ ]:
include("../src/DSI.jl")
include("../src/Zono_utils.jl")
include("../src/PZono.jl")
include("../src/DSZ.jl")
include("../src/propagation.jl")

In [ ]:
using PyPlot

## Specifying input ranges, networks and properties

In [ ]:
# from https://github.com/stanleybak/vnncomp2021/blob/main/benchmarks/acasxu/generate.py

init_lb_prop_1_2 = [55947.691, -pi, -pi, 1145, 0]
init_ub_prop_1_2 = [60760, pi, pi, 1200, 60]
acas_input_1_2 = interval.(init_lb_prop_1_2,init_ub_prop_1_2)

init_lb_prop_3 = [1500, -0.06, 3.1, 980, 960]
init_ub_prop_3 = [1800, 0.06, pi, 1200, 1200]
acas_input_3 = interval.(init_lb_prop_3,init_ub_prop_3)
    
init_lb_prop_4 = [1500, -0.06, 0, 1000, 700]
init_ub_prop_4 = [1800, 0.06, 0, 1200, 800]
acas_input_4 = interval.(init_lb_prop_4,init_ub_prop_4)

init_lb_prop_5 = [250, 0.2, -3.141592, 100, 0]
init_ub_prop_5 = [400, 0.4, -3.141592 + 0.005, 400, 400]
acas_input_5 = interval.(init_lb_prop_5,init_ub_prop_5)

init_lb_prop_61 = [12000, 0.7, -3.141592, 100, 0]
init_ub_prop_61 = [62000, 3.141592, -3.141592 + 0.005, 1200, 1200]
acas_input_61 = interval.(init_lb_prop_61,init_ub_prop_61)

init_lb_prop_62 = [12000, -3.141592, -3.141592, 100, 0]
init_ub_prop_62 = [62000, -0.7, -3.141592 + 0.005, 1200, 1200]
acas_input_62 = interval.(init_lb_prop_62,init_ub_prop_62)

init_lb_prop_7 = [0, -3.141592, -3.141592, 100, 0]
init_ub_prop_7 = [60760, 3.141592, 3.141592, 1200, 1200]
acas_input_7 = interval.(init_lb_prop_7,init_ub_prop_7)

init_lb_prop_8 = [0, -3.141592, -0.1, 600, 600]
init_ub_prop_8 = [60760, -0.75*3.141592, 0.1, 1200, 1200]
acas_input_8 = interval.(init_lb_prop_8,init_ub_prop_8)

init_lb_prop_9 = [2000, -0.4, -3.141592, 100, 0]
init_ub_prop_9 = [7000, -0.14, -3.141592 + 0.01, 150, 150]
acas_input_9 = interval.(init_lb_prop_9,init_ub_prop_9)

#output labels = ['Clear of Conflict (COC)', 'Weak Left', 'Weak Right', 'Strong Left', 'Strong Right']


function get_spec(prop::Int64)
    if (prop == 2)
        desc = "Unsafe if COC is maximal"
        # Unsafe if y1 > y2 and y1 > y3 and y1 > y4 and y1 > y5
        mat = [[-1. 1. 0. 0. 0.]
               [-1. 0. 1. 0. 0.]
               [-1. 0. 0. 1. 0.]
               [-1. 0. 0. 0. 1.]]
        rhs = [0., 0., 0., 0.]
    elseif (prop == 3) || (prop == 4)
        desc = "Unsafe if COC is minimal"
        mat = [[1. -1. 0. 0. 0.]
               [1. 0. -1. 0. 0.]
               [1. 0. 0. -1. 0.]
               [1. 0. 0. 0. -1.]]
        rhs = [0., 0., 0., 0.]
    end

    return (desc, mat, rhs)
end

mat_spec_2 = get_spec(2)[2]
rhs_spec_2 = get_spec(2)[3]
mat_spec_3_4 = get_spec(3)[2]
rhs_spec_3_4 = get_spec(3)[3]

mat_essai_1 = [[-1. 1. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]]

mat_essai_2 = [[0. 0. 0. 0. 0.]
[-1. 0. 1. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]]

mat_essai_3 = [[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[-1. 0. 0. 1. 0.]
[0. 0. 0. 0. 0.]]

mat_essai_4 = [[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[-1. 0. 0. 0. 1.]]


acas_nnet_1_2 = read_nnet("./ACASXU_networks/ACASXU_run2a_1_2_batch_2000.nnet", last_layer_activation = Id());
acas_nnet_1_3 = read_nnet("./ACASXU_networks/ACASXU_run2a_1_3_batch_2000.nnet", last_layer_activation = Id());
acas_nnet_1_4 = read_nnet("./ACASXU_networks/ACASXU_run2a_1_4_batch_2000.nnet", last_layer_activation = Id());
acas_nnet_1_5 = read_nnet("./ACASXU_networks/ACASXU_run2a_1_5_batch_2000.nnet", last_layer_activation = Id());
acas_nnet_1_6 = read_nnet("./ACASXU_networks/ACASXU_run2a_1_6_batch_2000.nnet", last_layer_activation = Id());
acas_nnet_2_2 = read_nnet("./ACASXU_networks/ACASXU_run2a_2_2_batch_2000.nnet", last_layer_activation = Id());
acas_nnet_2_9 = read_nnet("./ACASXU_networks/ACASXU_run2a_2_9_batch_2000.nnet", last_layer_activation = Id());
acas_nnet_3_1 = read_nnet("./ACASXU_networks/ACASXU_run2a_3_1_batch_2000.nnet", last_layer_activation = Id());
acas_nnet_3_6 = read_nnet("./ACASXU_networks/ACASXU_run2a_3_6_batch_2000.nnet", last_layer_activation = Id());
acas_nnet_3_7 = read_nnet("./ACASXU_networks/ACASXU_run2a_3_7_batch_2000.nnet", last_layer_activation = Id());
acas_nnet_4_1 = read_nnet("./ACASXU_networks/ACASXU_run2a_4_1_batch_2000.nnet", last_layer_activation = Id());
acas_nnet_4_7 = read_nnet("./ACASXU_networks/ACASXU_run2a_4_7_batch_2000.nnet", last_layer_activation = Id());
acas_nnet_5_3 = read_nnet("./ACASXU_networks/ACASXU_run2a_5_3_batch_2000.nnet", last_layer_activation = Id());
acas_nnet_1_7 = read_nnet("./ACASXU_networks/ACASXU_run2a_1_7_batch_2000.nnet", last_layer_activation = Id());
acas_nnet_1_9 = read_nnet("./ACASXU_networks/ACASXU_run2a_1_9_batch_2000.nnet", last_layer_activation = Id());


# Prop 2: x0 >= 0.6
# x0 <= 0.6798577687
# x1 >= -0.5
# x1 <= 0.5
# x2 >= -0.5
# x2 <= 0.5
# x3 >= 0.45
# x3 <= 0.5
# x4 >= -0.5
# x4 <= -0.45
# +y0 -y1 >= 0
# +y0 -y2 >= 0
# +y0 -y3 >= 0
# +y0 -y4 >= 0

init_lb_prop_1_2 = [0.6, -0.5, -0.5, 0.45, -0.5]
init_ub_prop_1_2 = [0.6798577687, 0.5, 0.5, 0.5, -0.45]
acas_input_1_2 = interval.(init_lb_prop_1_2,init_ub_prop_1_2)

# Prop 3:
# x0 >= -0.3035311561
# x0 <= -0.2985528119
# x1 >= -0.0095492966
# x1 <= 0.0095492966
# x2 >= 0.4933803236
# x2 <= 0.5
# x3 >= 0.3
# x3 <= 0.5
# x4 >= 0.3
# x4 <= 0.5
# +y0 -y1 <= 0
# +y0 -y2 <= 0
# +y0 -y3 <= 0
# +y0 -y4 <= 0

init_lb_prop_3 = [-0.3035311561, -0.0095492966, 0.4933803236, 0.3, 0.3]
init_ub_prop_3 = [-0.2985528119, 0.0095492966, 0.5, 0.5, 0.5]
acas_input_3 = interval.(init_lb_prop_3,init_ub_prop_3)

# Prop4:
# x0 >= -0.3035311561
# x0 <= -0.2985528119
# x1 >= -0.0095492966
# x1 <= 0.0095492966
# x2 >= 0
# x2 <= 0
# x3 >= 0.3181818182
# x3 <= 0.5
# x4 >= 0.0833333333
# x4 <= 0.1666666667
# +y0 -y1 <= 0
# +y0 -y2 <= 0
# +y0 -y3 <= 0
# +y0 -y4 <= 0

init_lb_prop_4 = [-0.3035311561, -0.0095492966, 0.0, 0.3181818182, 0.0833333333]
init_ub_prop_4 = [-0.2985528119, 0.0095492966,0.0, 0.5, 0.1666666667]
acas_input_4 = interval.(init_lb_prop_4,init_ub_prop_4)


## Setup for empirical propagation

In [ ]:
means_inputs = (init_ub_prop_1_2 .+ init_lb_prop_1_2) ./ 2
std_inputs = (init_ub_prop_1_2 .- means_inputs) ./ 3
init_range = init_ub_prop_1_2 .- init_lb_prop_1_2
input_boxes_desc = [([means_inputs[i] - 0.001 * init_range[i], means_inputs[i] + 0.001 * init_range[i]], std_inputs[i]) for i in 1:5]
input_boxes = [normal(interval(input_boxes_desc[i][1][1], input_boxes_desc[i][1][2]), input_boxes_desc[i][2]) for i in 1:5]


Generate samples from input random vectors:

In [ ]:
psamples = gaussian_samples(input_boxes_desc, 1000, 0.00005)

Display how big each input covering is:

In [ ]:
for i in 1:length(psamples)
    println(size(psamples[i]))
end

## (Optional) setup multithreaded julia environment

Useful for running the below function. Restarting the notebook is needed after running.

In [ ]:
using IJulia
installkernel("Julia (11 threads)", env=Dict("JULIA_NUM_THREADS"=>"11"))
Threads.nthreads()

Check if it works:

In [ ]:
Threads.nthreads()

## Empirical propagation

Choose which network to run for.

In [ ]:
outps = propagate_gaussians_multhread(psamples, acas_nnet_5_3, 1000)

In [ ]:
mat1 = [-1 0; 0 -1]
a = [1,-1]
y = [0,0]
println(all(mat1 * a .< y))
println(mat1 * a)

In [ ]:
mat =[-1.0 1.0 0.0 0.0 0.0;
      -1.0 0.0 1.0 0.0 0.0;
      -1.0 0.0 0.0 1.0 0.0;
      -1.0 0.0 0.0 0.0 1.0]
mat_2 = [1. -1. 0. 0. 0.
     1.  0. -1. 0. 0.
     1.  0.  0. -1. 0.
     1.  0.  0.  0. -1.]
rhs = [0.0, 0.0, 0.0, 0.0]
x = check_property(outps, mat, rhs)

The code below can be ran to recreate all experiments at once, however the Julia kernel tends to die, probably due to running out of memory. 

In [ ]:
outps_1_6 = propagate_gaussians_multhread(psamples, acas_nnet_1_6, 1000)
outps_2_2 = propagate_gaussians_multhread(psamples, acas_nnet_2_2, 1000)
outps_2_9 = propagate_gaussians_multhread(psamples, acas_nnet_2_9, 1000)
outps_3_1 = propagate_gaussians_multhread(psamples, acas_nnet_3_1, 1000)
outps_3_6 = propagate_gaussians_multhread(psamples, acas_nnet_3_6, 1000)
outps_3_7 = propagate_gaussians_multhread(psamplespoint1, acas_nnet_3_7, 200)
outps_4_1 = propagate_gaussians_multhread(psamplespoint1, acas_nnet_4_1, 200)
outps_4_7 = propagate_gaussians_multhread(psamplespoint1, acas_nnet_4_7, 200)
outps_5_3 = propagate_gaussians_multhread(psamplespoint1, acas_nnet_5_3, 200)

In [ ]:
chprop1_6 = check_property(outps_1_6, mat, rhs)
println("1_6", chprop1_6)
chprop2_2 = check_property(outps_2_2, mat, rhs)
println("2_2", chprop2_2)
chprop2_9 = check_property(outps_2_9, mat, rhs)
println("2_9", chprop2_9)
chprop3_1 = check_property(outps_3_1, mat, rhs)
println("3_1", chprop3_1)
chprop3_6 = check_property(outps_3_6, mat, rhs)
println("3_6", chprop3_6)
chprop3_7 = check_property(outps_3_7, mat, rhs)
println(chprop3_7)
chprop4_1 = check_property(outps_4_1, mat, rhs)
println(chprop4_1)
chprop4_7 = check_property(outps_4_7, mat, rhs)
println(chprop4_7)
chprop5_3 = check_property(outps_5_3, mat, rhs)
println(chprop5_3)

## DSZ Analysis

In [ ]:
# Defining the number of focal element for each component of the input vector for Property 2
vect_nb_focal_elem = [5, 80, 50, 6, 5]
println("vect_nb_focal_elem for Property 2 = ",vect_nb_focal_elem,":")
# the true flag in init_pbox_Normal means truncating the focal elements to restrict the range to [lb,ub]
acas_inputpbox_1_2 = init_pbox_Normal(init_lb_prop_1_2,init_ub_prop_1_2,vect_nb_focal_elem,true)


In [ ]:
@time vec_proba = dsz_approximate_nnet_and_condition_nostorage(acas_nnet_1_6, acas_inputpbox_1_2, mat_spec_2,rhs_spec_2) 
println("Property 2, net-1-6 : ", vec_proba[length(vec_proba)])


@time vec_proba = dsz_approximate_nnet_and_condition_nostorage(acas_nnet_2_2, acas_inputpbox_1_2, mat_spec_2,rhs_spec_2)
println("Property 2, net-2-2 : ", vec_proba[length(vec_proba)])


@time vec_proba = dsz_approximate_nnet_and_condition_nostorage(acas_nnet_2_9, acas_inputpbox_1_2, mat_spec_2,rhs_spec_2) 
println("Property 2, net-2-9 : ", vec_proba[length(vec_proba)])


@time vec_proba = dsz_approximate_nnet_and_condition_nostorage(acas_nnet_3_1, acas_inputpbox_1_2, mat_spec_2,rhs_spec_2) 
println("Property 2, net-3-1 : ", vec_proba[length(vec_proba)])


@time vec_proba = dsz_approximate_nnet_and_condition_nostorage(acas_nnet_3_6, acas_inputpbox_1_2, mat_spec_2,rhs_spec_2) 
println("Property 2, net-3-6 : ", vec_proba[length(vec_proba)])


@time vec_proba = dsz_approximate_nnet_and_condition_nostorage(acas_nnet_3_7, acas_inputpbox_1_2, mat_spec_2,rhs_spec_2)
println("Property 2, net-3-7 : ", vec_proba[length(vec_proba)])


@time vec_proba = dsz_approximate_nnet_and_condition_nostorage(acas_nnet_4_1, acas_inputpbox_1_2, mat_spec_2,rhs_spec_2)
println("Property 2, net-4-1 : ", vec_proba[length(vec_proba)])


@time vec_proba = dsz_approximate_nnet_and_condition_nostorage(acas_nnet_4_7, acas_inputpbox_1_2, mat_spec_2,rhs_spec_2)
println("Property 2, net-4-7 : ", vec_proba[length(vec_proba)])


@time vec_proba = dsz_approximate_nnet_and_condition_nostorage(acas_nnet_5_3, acas_inputpbox_1_2, mat_spec_2,rhs_spec_2)
println("Property 2, net-5-3 : ", vec_proba[length(vec_proba)])
     

### Properties 3 and 4

In [ ]:
# Defining the number of focal element for each component of the input vector for Properties 3 and 4
vect_nb_focal_elem = [5, 20, 1, 6, 5]
println("vect_nb_focal_elem for Properties 3 and 4 = ",vect_nb_focal_elem,":")
acas_inputpbox_3 = init_pbox_Normal(init_lb_prop_3,init_ub_prop_3,vect_nb_focal_elem,true)


@time vec_proba = dsz_approximate_nnet_and_condition_nostorage(acas_nnet_1_7, acas_inputpbox_3, mat_spec_3_4,rhs_spec_3_4) 
println("Property 3, net-1-7 : ", vec_proba[length(vec_proba)])

In [ ]:
#vect_nb_focal_elem = [5, 20, 1, 6, 5] 
acas_inputpbox_4 = init_pbox_Normal(init_lb_prop_4,init_ub_prop_4,vect_nb_focal_elem,true)


@time vec_proba = dsz_approximate_nnet_and_condition_nostorage(acas_nnet_1_9, acas_inputpbox_4, mat_spec_3_4,rhs_spec_3_4) 
println("Property 4, net-1-9 : ", vec_proba[length(vec_proba)])

### Heuristic refinement/optimization of focal element 

In [ ]:
#@time dsz_focal_refinement(acas_nnet_1_6,init_lb_prop_1_2,init_ub_prop_1_2, mat_spec_2,rhs_spec_2 , true, 0.05)